In [2]:
import numpy as np 
import cv2 
import os
import re

In [8]:
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_word(names):
    text = '-'.join(names) if len(names) > 1 else names[0]
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")

In [24]:
dataset_dir = "Dataset" 

imgs = [] #list image matrix 
labels = []
header_labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(sub_folder)[0]))
            header_labels.append(normalize_word([folder]))
            descs.append(normalize_word([folder, sub_folder]))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)


[##########] folder : Dataset/Daging Babi/ ----> file : IMG_20220110_094744.jpg
[##########] folder : Dataset/Daging Kuda/ ----> file : IMG_20220110_110515.jpg


In [25]:
for i in set(zip (header_labels, labels)):
    print(i[1], "\t:", i[0])

DagingBabi 	: Dataset
DagingSapi 	: Dataset
DagingKuda 	: Dataset


In [26]:
cv2.imshow("test img", imgs[10])

In [27]:
imgs[8].shape

(146, 146)

In [28]:
len(labels)

365

In [29]:
from skimage.feature import greycomatrix, greycoprops
import pandas as pd 

In [30]:
glcm = greycomatrix(imgs[100], distances=[5], angles=[0], levels=256,
                    symmetric=True, normed=True)

In [31]:
print('dissimilarity \t: %.04f' % greycoprops(glcm, 'dissimilarity')[0, 0])
print('correlation \t: %.04f' % greycoprops(glcm, 'correlation')[0, 0])
print('homogeneity \t: %.04f' % greycoprops(glcm, 'homogeneity')[0, 0])
print('contrast \t: %.04f' % greycoprops(glcm, 'contrast')[0, 0])
print('ASM \t\t: %.04f' % greycoprops(glcm, 'ASM')[0, 0])
print('energy \t\t: %.04f' % greycoprops(glcm, 'energy')[0, 0])

dissimilarity 	: 14.6420
correlation 	: 0.5349
homogeneity 	: 0.1006
contrast 	: 495.0603
ASM 		: 0.0005
energy 		: 0.0221


In [32]:
def calc_glcm_feature(img, label, dists=[5], agls=[0], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    feature.append(greycoprops(glcm, 'dissimilarity')[0, 0])
    feature.append(greycoprops(glcm, 'correlation')[0, 0])
    feature.append(greycoprops(glcm, 'homogeneity')[0, 0])
    feature.append(greycoprops(glcm, 'contrast')[0, 0])
    feature.append(greycoprops(glcm, 'ASM')[0, 0])
    feature.append(greycoprops(glcm, 'energy')[0, 0])
    feature.append(label)
    
    return feature

In [33]:
glcm_features = []
for img, label in zip(imgs, labels): 
    glcm_features.append(calc_glcm_feature(img, label))

In [34]:
# Create the pandas DataFrame for GLCM features data
glcm_features_df = pd.DataFrame(glcm_features, 
                                columns = ['dissimilarity', 
                                           'correlation',
                                           'homogeneity',
                                           'contrast',
                                           'ASM',
                                           'energy',
                                           'label'])

In [1]:
glcm_features_df[89:110]

NameError: name 'glcm_features_df' is not defined

In [40]:
glcm_features_df.to_csv("dataset_glcm.csv")